In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import zscore

# Ensure the data folder exists and the file is present
data_path = '../data/togo-dapaong_qc.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError(f"File not found at {data_path}. Ensure 'togo-dapaong_qc' is in the 'data/' folder.")

# Load the dataset
df_togo = pd.read_csv(data_path)

# Task 2: Data Profiling, Cleaning & EDA

# 1. Summary Statistics & Missing-Value Report
print("Summary Statistics:")
summary_stats = df_togo.describe()
print(summary_stats)

print("\nMissing Values:")
missing_values = df_togo.isna().sum()
print(missing_values)

# Calculate percentage of missing values
missing_percentage = (df_togo.isna().sum() / len(df_togo)) * 100
print("\nColumns with >5% Missing Values:")
print(missing_percentage[missing_percentage > 5])

# 2. Outlier Detection & Basic Cleaning
# Compute Z-scores for key columns
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
df_togo_zscores = df_togo[key_columns].apply(zscore, nan_policy='omit')
outliers = (df_togo_zscores.abs() > 3).any(axis=1)
print(f"\nNumber of rows with outliers (|Z| > 3): {outliers.sum()}")

# Impute missing values in key columns with median
for col in key_columns:
    df_togo[col].fillna(df_togo[col].median(), inplace=True)

# Drop rows with missing values in other critical columns (if any)
df_togo.dropna(subset=['Tamb', 'RH', 'BP'], inplace=True)

# Export cleaned DataFrame
cleaned_path = '../data/togo_clean.csv'
df_togo.to_csv(cleaned_path, index=False)
print(f"\nCleaned DataFrame exported to {cleaned_path}")

# 3. Time Series Analysis
# Convert Timestamp to datetime
df_togo['Timestamp'] = pd.to_datetime(df_togo['Timestamp'])

# Plot GHI, DNI, DHI, Tamb vs. Timestamp
plt.figure(figsize=(12, 8))
for col in ['GHI', 'DNI', 'DHI', 'Tamb']:
    plt.plot(df_togo['Timestamp'], df_togo[col], label=col)
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('Time Series of GHI, DNI, DHI, and Tamb')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Monthly trends
df_togo['Month'] = df_togo['Timestamp'].dt.month
monthly_avg = df_togo.groupby('Month')[['GHI', 'DNI', 'DHI']].mean()
monthly_avg.plot(kind='bar', figsize=(10, 6))
plt.title('Monthly Average GHI, DNI, DHI')
plt.xlabel('Month')
plt.ylabel('Average Value (W/m²)')
plt.tight_layout()
plt.show()

# 4. Cleaning Impact
cleaning_impact = df_togo.groupby('Cleaning')[['ModA', 'ModB']].mean()
cleaning_impact.plot(kind='bar', figsize=(8, 6))
plt.title('Average ModA and ModB Pre/Post Cleaning')
plt.xlabel('Cleaning (0 = No, 1 = Yes)')
plt.ylabel('Average Value (W/m²)')
plt.tight_layout()
plt.show()

# 5. Correlation & Relationship Analysis
# Correlation heatmap
corr_matrix = df_togo[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

# Scatter plots
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.scatter(df_togo['WS'], df_togo['GHI'], alpha=0.5)
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('GHI (W/m²)')
plt.title('WS vs. GHI')

plt.subplot(1, 3, 2)
plt.scatter(df_togo['WSgust'], df_togo['GHI'], alpha=0.5)
plt.xlabel('Wind Gust (m/s)')
plt.title('WSgust vs. GHI')

plt.subplot(1, 3, 3)
plt.scatter(df_togo['RH'], df_togo['Tamb'], alpha=0.5)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('Tamb (°C)')
plt.title('RH vs. Tamb')
plt.tight_layout()
plt.show()

# 6. Wind & Distribution Analysis
# Histogram for GHI
plt.figure(figsize=(8, 6))
plt.hist(df_togo['GHI'], bins=30, edgecolor='black')
plt.title('Histogram of GHI')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Histogram for WS
plt.figure(figsize=(8, 6))
plt.hist(df_togo['WS'], bins=30, edgecolor='black')
plt.title('Histogram of Wind Speed')
plt.xlabel('WS (m/s)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# 7. Temperature Analysis
plt.figure(figsize=(8, 6))
plt.scatter(df_togo['RH'], df_togo['GHI'], alpha=0.5)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('GHI (W/m²)')
plt.title('RH vs. GHI')
plt.tight_layout()
plt.show()

# 8. Bubble Chart
plt.figure(figsize=(10, 6))
plt.scatter(df_togo['Tamb'], df_togo['GHI'], s=df_togo['RH']*10, alpha=0.5)
plt.xlabel('Ambient Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs. Tamb (Bubble Size = RH)')
plt.tight_layout()
plt.show()

# Note: Wind rose plot requires additional libraries (e.g., windrose). Install if needed:
# pip install windrose
# Example wind rose (uncomment if installed):
"""
from windrose import WindroseAxes
ax = WindroseAxes.from_ax()
ax.bar(df_togo['WD'], df_togo['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title('Wind Rose Plot')
plt.show()
"""